In [1]:
from chromadb.config import Settings
from chromadb import Client
from langchain.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


In [2]:
# client = Client(Settings(
#     persist_directory="/Users/c360/Documents/chroma_db",  # Adjust to your storage path
#     # chroma_db_impl="duckdb+parquet",
# ))


In [3]:

swagger_file = "/Users/c360/Desktop/openapi.json"
loader = JSONLoader(
    file_path=swagger_file,
    jq_schema='.paths | to_entries[] | {endpoint: .key, methods: .value | to_entries[] | {method: .key, details: .value}}',
    text_content=False
)

In [4]:
docs = loader.load()


In [5]:
docs

[Document(metadata={'source': '/Users/c360/Desktop/openapi.json', 'seq_num': 1}, page_content='{"endpoint": "/pet", "methods": {"method": "put", "details": {"tags": ["pet"], "summary": "Update an existing pet", "description": "Update an existing pet by Id", "operationId": "updatePet", "requestBody": {"description": "Update an existent pet in the store", "content": {"application/json": {"schema": {"$ref": "#/components/schemas/Pet"}}, "application/xml": {"schema": {"$ref": "#/components/schemas/Pet"}}, "application/x-www-form-urlencoded": {"schema": {"$ref": "#/components/schemas/Pet"}}}, "required": true}, "responses": {"200": {"description": "Successful operation", "content": {"application/json": {"schema": {"$ref": "#/components/schemas/Pet"}}, "application/xml": {"schema": {"$ref": "#/components/schemas/Pet"}}}}, "400": {"description": "Invalid ID supplied"}, "404": {"description": "Pet not found"}, "422": {"description": "Validation exception"}}, "security": [{"petstore_auth": ["wr

In [6]:
docs[0]

Document(metadata={'source': '/Users/c360/Desktop/openapi.json', 'seq_num': 1}, page_content='{"endpoint": "/pet", "methods": {"method": "put", "details": {"tags": ["pet"], "summary": "Update an existing pet", "description": "Update an existing pet by Id", "operationId": "updatePet", "requestBody": {"description": "Update an existent pet in the store", "content": {"application/json": {"schema": {"$ref": "#/components/schemas/Pet"}}, "application/xml": {"schema": {"$ref": "#/components/schemas/Pet"}}, "application/x-www-form-urlencoded": {"schema": {"$ref": "#/components/schemas/Pet"}}}, "required": true}, "responses": {"200": {"description": "Successful operation", "content": {"application/json": {"schema": {"$ref": "#/components/schemas/Pet"}}, "application/xml": {"schema": {"$ref": "#/components/schemas/Pet"}}}}, "400": {"description": "Invalid ID supplied"}, "404": {"description": "Pet not found"}, "422": {"description": "Validation exception"}}, "security": [{"petstore_auth": ["wri

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function =len,)

chunks = text_splitter.split_documents(docs)
len(chunks)

19

In [8]:
chunks

[Document(metadata={'source': '/Users/c360/Desktop/openapi.json', 'seq_num': 1}, page_content='{"endpoint": "/pet", "methods": {"method": "put", "details": {"tags": ["pet"], "summary": "Update an existing pet", "description": "Update an existing pet by Id", "operationId": "updatePet", "requestBody": {"description": "Update an existent pet in the store", "content": {"application/json": {"schema": {"$ref": "#/components/schemas/Pet"}}, "application/xml": {"schema": {"$ref": "#/components/schemas/Pet"}}, "application/x-www-form-urlencoded": {"schema": {"$ref": "#/components/schemas/Pet"}}}, "required": true}, "responses": {"200": {"description": "Successful operation", "content": {"application/json": {"schema": {"$ref": "#/components/schemas/Pet"}}, "application/xml": {"schema": {"$ref": "#/components/schemas/Pet"}}}}, "400": {"description": "Invalid ID supplied"}, "404": {"description": "Pet not found"}, "422": {"description": "Validation exception"}}, "security": [{"petstore_auth": ["wr

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/Users/c360/work/rag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
vector_store =Chroma(
    collection_name="pets2",
    embedding_function=embeddings,
    persist_directory="/Users/c360/Documents/chroma_db"
)

In [9]:
vector_store.add_documents(documents=docs)

['7e3a6d45-71cb-4b29-afba-3b552f7dbfbf',
 '027b671a-3846-44ff-ac49-b8ba7d022bd5',
 '9ba23737-7abd-4814-96da-31d1d4070f72',
 '4c11a6a8-daff-491c-89da-3429f88be90d',
 '5039a7ae-e942-4f51-8579-3a30da96d1f1',
 '60a9399e-9079-443d-a3ee-13073357e666',
 '2c991d73-a18a-4bb9-a1c8-8611a7766cd0',
 '44cc3afb-444d-49b4-84e4-9fcf53123a68',
 '464123ff-a16f-4ac1-8073-61ed823eaee9',
 '5d1e0f4a-eb6c-4c23-a800-165fb0111393',
 '56029be8-73e0-4970-8991-c7b795881d20',
 'e3cc8a5f-a757-4a0b-bcbb-0e99d735b4bb',
 'b89527d9-43ba-4db7-8a33-7aeb5a43661e',
 'f2c67781-254c-4739-99e2-611d3ca76aed',
 '1453605d-ac8d-4a69-8a4b-bab50edbf8df',
 '44623c58-1cb5-41bc-8924-f28f1faca613',
 '6a42d28a-16c1-462d-b328-f8646ad4c4a0',
 'bf11dcbd-d642-4080-9299-bb1fbc1933ba',
 '71d48437-29b7-47bc-a5fe-25c063a23bfc']

In [35]:
endpoints = []

# Iterate through the Document objects
for doc in data:
    try:
        # Parse the JSON content from page_content
        content = json.loads(doc.page_content)
        endpoint = content.get("endpoint")
        if endpoint:  # Add to the list if the endpoint exists
            endpoints.append(endpoint)
    except json.JSONDecodeError as e:
        print(f"Error parsing content: {e}")

# Print all the endpoints
print("Extracted Endpoints:")
for endpoint in endpoints:
    print(endpoint)

Extracted Endpoints:
/pet
/pet
/pet/findByStatus
/pet/findByTags
/pet/{petId}
/pet/{petId}
/pet/{petId}
/pet/{petId}/uploadImage
/store/inventory
/store/order
/store/order/{orderId}
/store/order/{orderId}
/user
/user/createWithList
/user/login
/user/logout
/user/{username}
/user/{username}
/user/{username}


In [21]:
with open("/Users/c360/Desktop/openapi.json", "r") as f:
    raw_json = json.load(f)
print("Raw JSON:", raw_json)


Raw JSON: {'openapi': '3.0.3', 'info': {'title': 'Swagger Petstore - OpenAPI 3.0', 'description': "This is a sample Pet Store Server based on the OpenAPI 3.0 specification.  You can find out more about\nSwagger at [https://swagger.io](https://swagger.io). In the third iteration of the pet store, we've switched to the design first approach!\nYou can now help us improve the API whether it's by making changes to the definition itself or to the code.\nThat way, with time, we can improve the API in general, and expose some of the new features in OAS3.\n\n_If you're looking for the Swagger 2.0/OAS 2.0 version of Petstore, then click [here](https://editor.swagger.io/?url=https://petstore.swagger.io/v2/swagger.yaml). Alternatively, you can load via the `Edit > Load Petstore OAS 2.0` menu option!_\n\nSome useful links:\n- [The Pet Store repository](https://github.com/swagger-api/swagger-petstore)\n- [The source API definition for the Pet Store](https://github.com/swagger-api/swagger-petstore/bl